# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')

In [6]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [7]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [8]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [9]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale

1.0

In [14]:
import numpy as np
from collections.abc import Iterable

In [15]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [16]:
_outputs_shape

120000

# Model's Layers definition

In [17]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [18]:
model_name = dataset_name+'LTAAE'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [19]:
from training.latent_adversarial.autoencoders.AAE import AAE as AE

In [20]:
inputs_shape=IMG_SIZE

In [21]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [22]:
from utils.data_and_files.file_utils import create_if_not_exist

In [23]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [24]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonAAE\\var_save_dir'

In [25]:
#to restore trained model, set filepath=_restore

In [ ]:
from stats.traditional_adversarial_losses import \
    create_latent_adversarial_real_losses, \
    create_latent_adversarial_fake_losses, \
    create_latent_adversarial_losses

latent_adversarial_losses = {
    'latent_adversarial_losses': create_latent_adversarial_losses,
    'latent_adversarial_real_losses': create_latent_adversarial_real_losses,
    'latent_adversarial_fake_losses': create_latent_adversarial_fake_losses,
}

In [26]:
ae = AE( 
    name=model_name,
    adversarial_losses=latent_adversarial_losses,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 32)      128       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 32)      1056      
_________________________________________________________________
flatten (Flatten)            (None, 1280000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                40960032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [27]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonAAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                40961344  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       3962124   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 44,923,468
Trainable params: 44,923,398
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonAAE\\csv_dir\\pokemonAAE.csv'

In [33]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [36]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)


training traditional basicAE
Train for 10 steps, validate for 10 steps


Epoch 1/100
10/10 [==============================] - ETA: 39s - loss: 61043.0938 - psnr: 7.2275 - ssmi: 0.0235 - sharp_diff: 6.421 - ETA: 18s - loss: 170882.1406 - psnr: 7.6767 - ssmi: 0.0137 - sharp_diff: 5.31 - ETA: 11s - loss: 207169.1562 - psnr: 7.8892 - ssmi: 0.0106 - sharp_diff: 4.95 - ETA: 7s - loss: 208578.6094 - psnr: 7.9392 - ssmi: 0.0090 - sharp_diff: 4.7719 - ETA: 5s - loss: 183658.0125 - psnr: 8.0393 - ssmi: 0.0084 - sharp_diff: 4.715 - ETA: 3s - loss: 167379.5495 - psnr: 8.0965 - ssmi: 0.0080 - sharp_diff: 4.678 - ETA: 2s - loss: 154346.1239 - psnr: 8.2133 - ssmi: 0.0077 - sharp_diff: 4.653 - ETA: 1s - loss: 145183.1182 - psnr: 8.2620 - ssmi: 0.0076 - sharp_diff: 4.634 - ETA: 0s - loss: 138042.0295 - psnr: 8.3012 - ssmi: 0.0074 - sharp_diff: 4.620 - 10s 992ms/step - loss: 131879.9242 - psnr: 8.3653 - ssmi: 0.0074 - sharp_diff: 4.6097 - val_loss: 70586.8441 - val_psnr: 9.2658 - val_ssmi: 0.1813 - val_sharp_diff: 12.6441


Epoch 2/100
10/10 [==============================] - ETA: 2s - loss: 82100.0234 - psnr: 8.6560 - ssmi: 0.0065 - sharp_diff: 4.49 - ETA: 2s - loss: 84363.3945 - psnr: 8.5429 - ssmi: 0.0062 - sharp_diff: 4.49 - ETA: 1s - loss: 84092.3698 - psnr: 8.5145 - ssmi: 0.0060 - sharp_diff: 4.49 - ETA: 1s - loss: 85677.5781 - psnr: 8.4391 - ssmi: 0.0060 - sharp_diff: 4.49 - ETA: 1s - loss: 84257.5672 - psnr: 8.4862 - ssmi: 0.0060 - sharp_diff: 4.49 - ETA: 1s - loss: 84044.5534 - psnr: 8.4987 - ssmi: 0.0060 - sharp_diff: 4.49 - ETA: 0s - loss: 84404.1585 - psnr: 8.4686 - ssmi: 0.0060 - sharp_diff: 4.49 - ETA: 0s - loss: 85460.8701 - psnr: 8.4081 - ssmi: 0.0060 - sharp_diff: 4.49 - ETA: 0s - loss: 84948.7335 - psnr: 8.4354 - ssmi: 0.0061 - sharp_diff: 4.49 - 4s 411ms/step - loss: 84664.9320 - psnr: 8.4649 - ssmi: 0.0062 - sharp_diff: 4.4989 - val_loss: 67068.8742 - val_psnr: 9.5662 - val_ssmi: 0.1831 - val_sharp_diff: 12.7089


Epoch 3/100
10/10 [==============================] - ETA: 2s - loss: 87921.7109 - psnr: 8.2077 - ssmi: 0.0062 - sharp_diff: 4.49 - ETA: 2s - loss: 82459.6562 - psnr: 8.5218 - ssmi: 0.0066 - sharp_diff: 4.50 - ETA: 1s - loss: 102941.8490 - psnr: 7.6978 - ssmi: 0.0064 - sharp_diff: 4.492 - ETA: 1s - loss: 98085.8887 - psnr: 7.8962 - ssmi: 0.0066 - sharp_diff: 4.497 - ETA: 1s - loss: 96678.6719 - psnr: 7.9229 - ssmi: 0.0065 - sharp_diff: 4.49 - ETA: 1s - loss: 94481.8997 - psnr: 8.0202 - ssmi: 0.0065 - sharp_diff: 4.49 - ETA: 0s - loss: 92725.5603 - psnr: 8.1110 - ssmi: 0.0066 - sharp_diff: 4.50 - ETA: 0s - loss: 91037.1221 - psnr: 8.2025 - ssmi: 0.0066 - sharp_diff: 4.50 - ETA: 0s - loss: 90472.3663 - psnr: 8.2311 - ssmi: 0.0066 - sharp_diff: 4.50 - 4s 414ms/step - loss: 90963.0617 - psnr: 8.2041 - ssmi: 0.0066 - sharp_diff: 4.5042 - val_loss: 65840.3750 - val_psnr: 9.5899 - val_ssmi: 0.1834 - val_sharp_diff: 12.7219


Epoch 4/100
10/10 [==============================] - ETA: 2s - loss: 73237.1328 - psnr: 8.9580 - ssmi: 0.0067 - sharp_diff: 4.51 - ETA: 2s - loss: 80524.9375 - psnr: 8.6486 - ssmi: 0.0063 - sharp_diff: 4.50 - ETA: 1s - loss: 79421.8620 - psnr: 8.7213 - ssmi: 0.0064 - sharp_diff: 4.51 - ETA: 1s - loss: 79320.8945 - psnr: 8.6967 - ssmi: 0.0062 - sharp_diff: 4.50 - ETA: 1s - loss: 78544.6219 - psnr: 8.7176 - ssmi: 0.0061 - sharp_diff: 4.50 - ETA: 1s - loss: 77331.7982 - psnr: 8.7717 - ssmi: 0.0061 - sharp_diff: 4.51 - ETA: 0s - loss: 78619.7210 - psnr: 8.7182 - ssmi: 0.0063 - sharp_diff: 4.51 - ETA: 0s - loss: 79093.6758 - psnr: 8.6793 - ssmi: 0.0062 - sharp_diff: 4.51 - ETA: 0s - loss: 80080.3108 - psnr: 8.6281 - ssmi: 0.0063 - sharp_diff: 4.50 - 4s 401ms/step - loss: 79772.8859 - psnr: 8.6413 - ssmi: 0.0063 - sharp_diff: 4.5099 - val_loss: 69543.1422 - val_psnr: 9.3239 - val_ssmi: 0.1811 - val_sharp_diff: 12.6592


Epoch 5/100
10/10 [==============================] - ETA: 2s - loss: 74357.2031 - psnr: 8.9028 - ssmi: 0.0065 - sharp_diff: 4.52 - ETA: 2s - loss: 81323.2109 - psnr: 8.6107 - ssmi: 0.0068 - sharp_diff: 4.51 - ETA: 1s - loss: 81339.3906 - psnr: 8.5962 - ssmi: 0.0068 - sharp_diff: 4.51 - ETA: 1s - loss: 81435.9746 - psnr: 8.5776 - ssmi: 0.0067 - sharp_diff: 4.51 - ETA: 1s - loss: 81345.8906 - psnr: 8.5688 - ssmi: 0.0067 - sharp_diff: 4.51 - ETA: 1s - loss: 80843.1641 - psnr: 8.5940 - ssmi: 0.0067 - sharp_diff: 4.52 - ETA: 0s - loss: 76853.6568 - psnr: 8.7810 - ssmi: 0.0067 - sharp_diff: 4.52 - ETA: 0s - loss: 77323.8677 - psnr: 8.7408 - ssmi: 0.0066 - sharp_diff: 4.52 - ETA: 0s - loss: 77972.0456 - psnr: 8.7044 - ssmi: 0.0066 - sharp_diff: 4.52 - 4s 405ms/step - loss: 78628.3160 - psnr: 8.6687 - ssmi: 0.0066 - sharp_diff: 4.5209 - val_loss: 66687.3973 - val_psnr: 9.5867 - val_ssmi: 0.1826 - val_sharp_diff: 12.6788


Epoch 6/100
10/10 [==============================] - ETA: 2s - loss: 85564.9688 - psnr: 8.4768 - ssmi: 0.0072 - sharp_diff: 4.53 - ETA: 2s - loss: 88739.4141 - psnr: 8.2679 - ssmi: 0.0070 - sharp_diff: 4.52 - ETA: 1s - loss: 86262.7422 - psnr: 8.4049 - ssmi: 0.0070 - sharp_diff: 4.52 - ETA: 1s - loss: 84080.8984 - psnr: 8.5094 - ssmi: 0.0071 - sharp_diff: 4.52 - ETA: 1s - loss: 82045.6656 - psnr: 8.5787 - ssmi: 0.0070 - sharp_diff: 4.53 - ETA: 1s - loss: 83049.6445 - psnr: 8.5217 - ssmi: 0.0069 - sharp_diff: 4.53 - ETA: 0s - loss: 81774.3705 - psnr: 8.5829 - ssmi: 0.0069 - sharp_diff: 4.53 - ETA: 0s - loss: 80946.3252 - psnr: 8.6112 - ssmi: 0.0069 - sharp_diff: 4.53 - ETA: 0s - loss: 80442.0946 - psnr: 8.6368 - ssmi: 0.0069 - sharp_diff: 4.53 - 5s 502ms/step - loss: 80569.5320 - psnr: 8.6105 - ssmi: 0.0069 - sharp_diff: 4.5354 - val_loss: 70399.6895 - val_psnr: 9.2908 - val_ssmi: 0.1821 - val_sharp_diff: 12.6206


Epoch 7/100
 9/10 [==========================>...] - ETA: 2s - loss: 80938.2500 - psnr: 8.5491 - ssmi: 0.0068 - sharp_diff: 4.54 - ETA: 2s - loss: 83108.3984 - psnr: 8.4653 - ssmi: 0.0069 - sharp_diff: 4.54 - ETA: 2s - loss: 81964.2891 - psnr: 8.5095 - ssmi: 0.0068 - sharp_diff: 4.54 - ETA: 1s - loss: 81487.0371 - psnr: 8.5268 - ssmi: 0.0070 - sharp_diff: 4.54 - ETA: 1s - loss: 79462.1438 - psnr: 8.6207 - ssmi: 0.0070 - sharp_diff: 4.54 - ETA: 1s - loss: 79516.8242 - psnr: 8.6221 - ssmi: 0.0071 - sharp_diff: 4.55 - ETA: 0s - loss: 79562.7143 - psnr: 8.6109 - ssmi: 0.0071 - sharp_diff: 4.55 - ETA: 0s - loss: 79639.6143 - psnr: 8.6035 - ssmi: 0.0072 - sharp_diff: 4.55 - ETA: 0s - loss: 79016.8229 - psnr: 8.6453 - ssmi: 0.0072 - sharp_diff: 4.5539WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,psnr,ssmi,sharp_diff


KeyError: 'val_loss'